In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
#parameters
vocab_size = 10000
maxlen = 200
embed_dim =32
num_heads = 2
ff_dim = 32

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train,maxlen=maxlen)
x_test = pad_sequences(x_test,maxlen=maxlen)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 45s 3us/step


In [15]:
inputs = layers.Input(shape=(maxlen,))

In [5]:
embedding_layer = layers.Embedding(vocab_size,embed_dim)(inputs)

In [6]:
positions = tf.range(start=0, limit=maxlen, delta=1)
position_embeddings = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)(positions)
x = embedding_layer + position_embeddings

In [10]:
attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)

In [12]:
x = layers.LayerNormalization()(x+attention)